In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('final_data.xlsx',index_col=0)

In [3]:
data

,url,id,name,Artist,key_word,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger,empty_a,sadness_a,enthusiasm_a,worry_a,love_a,fun_a,hate_a,happiness_a,boredom_a,relief_a,anger_a,gerne
0,https://open.spotify.com/track/0eFMbKCRw8KByXy...,0eFMbKCRw8KByXyWBw8WO7,Still With You,Jung Kook,face want see eye tell ecstatic memory even da...,0.016461,0.265974,0.067839,0.144616,0.116033,0.206217,0.015079,0.134595,0.002582,0.028249,0.002356,0.000225,0.000191,0.000152,1.000000,0.000023,0.000135,1.214844e-05,0.000221,0.000104,0.000052,1.310354e-05,['k-pop']
1,https://open.spotify.com/track/7eJMfftS33KTjuF...,7eJMfftS33KTjuF7lTsMCx,death bed (coffee for your head),Powfu,stay awake long go bed make cup coffee head ge...,0.122187,0.059084,0.039108,0.113228,0.024429,0.023013,0.015683,0.212476,0.004940,0.384384,0.001468,0.000062,0.000079,0.001033,0.000177,0.000280,0.001177,2.478030e-04,0.001157,0.999988,0.000124,2.485695e-04,"['sad lo-fi', 'sad rap']"
2,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,2CbGuO0LtVvbh3umN3mDwM,Nan-Nan,Fujii Kaze,say didnt ask anything face w swear corner flo...,0.005816,0.248961,0.003242,0.483580,0.044740,0.008447,0.022421,0.152159,0.000405,0.029745,0.000483,0.000284,0.000221,0.000088,0.000382,0.000073,0.000093,1.870017e-05,0.000099,0.000147,1.000000,1.633500e-05,['japanese teen pop']
3,https://open.spotify.com/track/1JRNrJKTQhfvxII...,1JRNrJKTQhfvxII8kkIcAQ,Mo-Eh-Wa,Fujii Kaze,night blown cold wind dont wander blow inner w...,0.077518,0.249271,0.011364,0.406069,0.011648,0.038887,0.005062,0.123563,0.002235,0.072786,0.001598,0.000252,0.000261,0.000068,0.000342,0.000151,0.000067,1.677677e-05,0.000076,0.000135,0.999990,1.130652e-05,['japanese teen pop']
4,https://open.spotify.com/track/57u1FaZlztApkcT...,57u1FaZlztApkcT5x5XNuD,YASASHISA,Fujii Kaze,dream dream shade person kill kindness dream s...,0.001735,0.235752,0.001410,0.463885,0.142529,0.009176,0.010140,0.120540,0.000858,0.013238,0.000737,0.000165,0.000172,0.000190,0.000034,0.000072,0.000203,1.000000e+00,0.000166,0.000486,0.000042,1.000000e+00,['japanese teen pop']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,https://open.spotify.com/track/74xCxQy50bkpvOJ...,74xCxQy50bkpvOJm5MgD5P,Murder in the City,Various Artists,get murder city revenge name one person dead p...,0.011245,0.162807,0.006966,0.456847,0.082303,0.040452,0.002504,0.231635,0.000722,0.004143,0.000376,1.000000,1.000000,0.000048,0.000154,1.000000,0.000033,4.020569e-05,0.000032,0.000517,0.000025,5.175872e-05,"['art pop', 'pop']"
8516,https://open.spotify.com/track/4FhQ9BYFO3yFAAS...,4FhQ9BYFO3yFAASazZdvRX,September Song - Guitar Acoustic,Various Artists,innocent darling use talk til morning mixtape ...,0.021068,0.348507,0.035223,0.432478,0.046740,0.014127,0.007841,0.074129,0.001142,0.017115,0.001629,0.015458,0.015540,0.000100,0.000243,0.016848,0.000087,9.876737e-01,0.000068,0.000284,0.000351,9.881506e-01,"['art pop', 'pop']"
8517,https://open.spotify.com/track/70RiiOJdklpSmLS...,70RiiOJdklpSmLSBnKlcYF,Control,Various Artists,wan na lose control even though old know shake...,0.002924,0.170607,0.004646,0.211214,0.003779,0.007508,0.575163,0.020068,0.000557,0.002283,0.001252,1.000000,1.000000,0.000298,0.000185,1.000000,0.000290,1.177873e-04,0.000280,0.000078,0.000063,1.206514e-04,"['art pop', 'pop']"
8518,https://open.spotify.com/track/26Ow8vxj4n0uew1...,26Ow8vxj4n0uew1UP8CU56,Heart's Content - Strings Mix,Various Artists,love live without u one thing know sure love f...,0.004582,0.297325,0.004344,0.583962,0.050943,0.004454,0.001543,0.051992,0.000209,0.000479,0.000167,1.000000,1.000000,0.000120,0.000130,1.000000,0.000105,1.357035e-04,0.000088,0.000229,0.000052,1.641192e-04,"['art pop', 'pop']"


In [4]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
from itertools import islice

import requests
import json

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

## 도서 정보 들고오기

In [6]:
rest_api_key = "41d651c93152d5ec054dc828cacfa671"
url = "https://dapi.kakao.com/v3/search/book"
header = {"authorization": "KakaoAK "+rest_api_key}

In [7]:
#궁금한 도서의 isbn 입력
querynum = {"query": '어린왕자'}

In [8]:
#도서 정보 불러오기
response = requests.get(url, headers=header, params = querynum)
content = response.text
책정보 = json.loads(content)['documents'][0]

In [9]:
책정보

{'authors': ['생텍쥐페리'],
 'contents': '다른 별에서 온 어린 왕자의 순수한 시선으로 모순된 어른들의 세계를 비추는 전 세계가 사랑하는 아름다운 이야기 『어린 왕자』. 그동안 프랑스어 원문에 대한 섬세한 이해, 정확하고도 아름다운 문장력, 예리한 문학적 통찰을 고루 갖춘 번역으로 문학 번역에서 큰 입지를 굳혀 온 황현산. 그는 이 작품을 새롭게 번역하면서 생텍쥐페리의 진솔한 문체를 고스란히 살려 내기 위해 심혈을 기울였고, 원전의 가치를 충실히 살린 한국어 결정판을 마련하고자 했다.    이 작품',
 'datetime': '2015-10-20T00:00:00.000+09:00',
 'isbn': '8932917248 9788932917245',
 'price': 11800,
 'publisher': '열린책들',
 'sale_price': 10620,
 'status': '정상판매',
 'thumbnail': 'https://search1.kakaocdn.net/thumb/R120x174.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flbook%2Fimage%2F507587%3Ftimestamp%3D20231007143853',
 'title': '어린 왕자',
 'translators': ['황현산'],
 'url': 'https://search.daum.net/search?w=bookpage&bookId=507587&q=%EC%96%B4%EB%A6%B0+%EC%99%95%EC%9E%90'}

In [10]:
책정보['url']

'https://search.daum.net/search?w=bookpage&bookId=507587&q=%EC%96%B4%EB%A6%B0+%EC%99%95%EC%9E%90'

In [11]:
book = pd.DataFrame({'title': 책정보['title'],
              'isbn': 책정보['isbn'],
              'authors': 책정보['authors'],
              'publisher': 책정보['publisher']})
book

,title,isbn,authors,publisher
0,어린 왕자,8932917248 9788932917245,생텍쥐페리,열린책들


In [12]:
title = book['title']
title[0]

'어린 왕자'

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import time

In [14]:
target_url = 책정보['url']
# 옵션 생성
options = webdriver.ChromeOptions()
# 창 숨기는 옵션 추가
options.add_argument("headless")

driver = webdriver.Chrome(options=options)
driver.get(target_url)
time.sleep(5)

try :
    botton = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[2]/div[3]/a')
    botton.click()
except :
    pass
책소개 = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[2]/p')

time.sleep(3)
try :
    botton = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[5]/div[3]/a')
    botton.click()
except :
    pass
책속으로 = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[5]/p')

time.sleep(3)
try :
    botton = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[6]/div[3]/a')
    botton.click()
except :
    pass
서평 = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[6]/p')

book['책소개'] = 책소개.text
book['책속으로'] = 책속으로.text
book['서평'] = 서평.text


In [15]:
img= driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[1]/div[1]/span/img')

In [16]:
img_src = img.get_attribute('src')
img_src

'https://search1.kakaocdn.net/thumb/C116x164.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flbook%2Fimage%2F507587%3Ftimestamp%3D20231007143853%3Fmoddttm=202310200333'

In [17]:
driver.close()

In [18]:
book

,title,isbn,authors,publisher,책소개,책속으로,서평
0,어린 왕자,8932917248 9788932917245,생텍쥐페리,열린책들,다른 별에서 온 어린 왕자의 순수한 시선으로 모순된 어른들의 세계를 비추는 전 세계...,"「내 생활은 단조로워. 나는 닭을 쫓고, 사람들은 나를 쫓고. 닭들은 모두 그게 그...","전 세계가 사랑하는 가장 아름다운 이야기, 생텍쥐페리의 『어린 왕자』가 문학 평론가..."


In [19]:
book['책소개'][0]

'다른 별에서 온 어린 왕자의 순수한 시선으로 모순된 어른들의 세계를 비추는 전 세계가 사랑하는 아름다운 이야기 『어린 왕자』. 그동안 프랑스어 원문에 대한 섬세한 이해, 정확하고도 아름다운 문장력, 예리한 문학적 통찰을 고루 갖춘 번역으로 문학 번역에서 큰 입지를 굳혀 온 황현산. 그는 이 작품을 새롭게 번역하면서 생텍쥐페리의 진솔한 문체를 고스란히 살려 내기 위해 심혈을 기울였고, 원전의 가치를 충실히 살린 한국어 결정판을 마련하고자 했다.\n\n\n\n이 작품은 어떤 소설보다도 독자들에게 오래 기억되며 마음을 움직이는 힘을 가진 문장들로 가득하다. 역자 황현산은 그 힘의 근원을 저자 생텍쥐페리의 진솔하고 열정적인 문체라고 풀이했다. 꾸밈없는 진솔한 문체와 동화처럼 단순해 보이는 이야기 속에 삶을 돌아보는 깊은 성찰을 아름다운 은유로 녹여 내 깊은 여운을 주는 이 작품을 보다 새롭고 완성도 높은 번역으로 다시 한 번 음미하며 읽어 볼 수 있다.'

In [20]:
book['책속으로'][0]

'「내 생활은 단조로워. 나는 닭을 쫓고, 사람들은 나를 쫓고. 닭들은 모두 그게 그거고, 사람들도 모두 그게 그거고. 그래서 난 좀 지겨워. 그러나 네가 날 길들인다면 내 생활은 햇빛을 받은 듯 환해질 거야. 모든 발자국 소리와는 다르게 들릴 발자국 소리를 나는 듣게 될 거야. 다른 발자국 소리는 나를 땅속에 숨게 하지. 네 발자국 소리는 음악처럼 나를 굴 밖으로 불러낼 거야. 그리고 저기, 밀밭이 보이지? 나는 빵을 먹지 않아! 밀은 내게 아무 소용이 없어. 그래서 슬퍼! 그러나 네 머리칼은 금빛이야. 그래서 네가 나를 길들인다면 정말 놀라운 일이 일어날 거야. 밀은, 금빛이어서, 너를 생각나게 할 거야. 그래서 나는 밀밭에 스치는 바람 소리를 사랑하게 될 거고…….」\n-본문 86면\n\n「가령 오후 4시에 네가 온다면 나는 3시부터 행복해지기 시작할 거야. 시간이 갈수록 난 더 행복해질 거야. 4시가 되면, 벌써, 나는 안달이 나서 안절부절못하게 될 거야. 난 행복의 대가가 무엇인지 알게 될 거야! 그러나 네가 아무 때나 온다면, 몇 시에 마음을 준비해야 할지 알 수 없을 거야……. 의례가 필요해.」\n「의례가 뭐야?」 어린 왕자가 말했다.\n「그것도 모두들 너무 잊고 있는 것이지.」 여우가 말했다. 「그건 어떤 날을 다른 날과 다르게, 어떤 시간을 다른 시간과 다르게 만드는 거야. 이를테면 사냥꾼들에게도 의례가 있지. 그들은 목요일이면 마을 처녀들하고 춤을 춘단다. 그래서 목요일은 경이로운 날이지! 나는 포도밭까지 산책을 나가지. 만일에 사냥꾼들이 아무 때나 춤을 춘다면 모든 날이 다 그게 그거고, 내게는 휴일이 없을 거야.」\n-본문 87면\n\n어린 왕자는 장미들을 다시 보러 갔다.\n그는 꽃들에게 말했다. 「너희들은 내 장미를 전혀 닮지 않았어, 너희들은 아직 아무것도 아니야. 누구도 너희들을 길들이지 않았고, 너희들은 누구도 길들이지 않았어. 너희들은 옛날 내 여우와 같아. 수많은 다른 여우들과 다를 게 없는 여우 한 마리에 지나지 않았지. 그

In [21]:
book['서평'][0]

'전 세계가 사랑하는 가장 아름다운 이야기, 생텍쥐페리의 『어린 왕자』가 문학 평론가 황현산 선생의 번역으로 열린책들에서 출간되었다. 프랑스어 원문에 대한 섬세한 이해, 정확하고도 아름다운 문장력, 예리한 문학적 통찰을 고루 갖춘 번역으로 문학 번역에서 큰 입지를 굳혀 온 황현산 선생은 이 작품을 새롭게 번역하면서 생텍쥐페리의 진솔한 문체를 고스란히 살려 내기 위해 심혈을 기울였다. 원문 텍스트 선택부터 번역의 마무리 작업까지, 국내에 출간된 많은 『어린 왕자』 중에서도 특히 원전의 가치를 충실히 살린 한국어 결정판을 마련하고자 했다. 다른 별에서 온 어린 왕자의 순수한 시선으로 모순된 어른들의 세계를 비추는 이 소설은, 꾸밈없는 진솔한 문체와 동화처럼 단순해 보이는 이야기 속에 삶을 돌아보는 깊은 성찰을 아름다운 은유로 녹여 낸 작품이다. 『어린 왕자』를 다시 읽을 때마다 우리는 삶에서 가장 중요한 것들, 그러나 잊히거나 상실된 것들, 《눈에 보이지 않는》 것들을 돌아보는 자리로 돌아오게 된다. 어린 시절 읽었던 이 작품을 보다 새롭고 완성도 높은 번역으로 다시 한 번 음미하며 읽어 볼 때다.\n\n생텍쥐페리의 진솔한 문체를 고스란히 살려 낸,\n문학 평론가 황현산의 번역으로 만나는 『어린 왕자』\n\n논문 같은 글은 논증 장치로 설득하지만, 시나 소설은 문체로 마음을 움직인다. 가령 『어린 왕자』에서 여우가 《자기가 길들인 것만 알 수 있는 거야》라고 말할 때, 이 말이 옳다는 증거는 어디에도 없다. 오직 저자 생텍쥐페리의 진솔하고 열정적인 문체만이 이 말의 진실성을 믿게 하고 우리를 감동하게 한다. - 황현산\n\n힘이 있는 문장들은 마음을 움직인다. 생텍쥐페리의 『어린 왕자』는 그 어떤 소설보다도 독자들에게 오래 기억되며 마음을 움직이는 힘을 가진 문장들로 가득하다. 그것은 그 문장들이 화려한 미사여구나 치밀한 논증으로 전하는 바를 설명하고 있어서가 아니다. 이번에 열린책들에서 출간되는 『어린 왕자』를 번역한 황현산 선생은 그 힘의 근원을 저자 생텍쥐페

In [22]:
import googletrans
from googletrans import Translator

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/seoeunseo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/seoeunseo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/seoeunseo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/seoeunseo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
#영어 불용어 사전
stops = set(stopwords.words('english'))

def hapus_url(text):
    mention_pattern = r'@[\w]+'
    cleaned_text = re.sub(mention_pattern, '', text)
    return re.sub(r'http\S+','', cleaned_text)

#특수문자 제거
#영어 대소문자, 숫자, 공백문자(스페이스, 탭, 줄바꿈 등) 아닌 문자들 제거
def remove_special_characters(text, remove_digits=True):
    text=re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text


#불용어 제거
def delete_stops(text):
    text = text.lower().split()
    text = ' '.join([word for word in text if word not in stops])
    return text
   
    
#품사 tag 매칭용 함수
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    

#품사 태깅 + 표제어 추출
def tockenize(text):
    tokens=word_tokenize(text)
    pos_tokens=nltk.pos_tag(tokens)
    
    text_t=list()
    for _ in pos_tokens:
        text_t.append([_[0], get_wordnet_pos(_[1])])
    
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word[0], word[1]) for word in text_t])
    return text



def clean(text):
    text = remove_special_characters(text, remove_digits=True)
    text = delete_stops(text)
    text = tockenize(text)
    return text

In [24]:
translator = Translator()
for col in ['책소개', '책속으로', '서평']:
    name = col+'_trans'
    if book[col].values == '':
        book[name] = ''
        continue
    book[name] = clean(translator.translate(hapus_url(book.loc[0, col])).text)

book

,title,isbn,authors,publisher,책소개,책속으로,서평,책소개_trans,책속으로_trans,서평_trans
0,어린 왕자,8932917248 9788932917245,생텍쥐페리,열린책들,다른 별에서 온 어린 왕자의 순수한 시선으로 모순된 어른들의 세계를 비추는 전 세계...,"「내 생활은 단조로워. 나는 닭을 쫓고, 사람들은 나를 쫓고. 닭들은 모두 그게 그...","전 세계가 사랑하는 가장 아름다운 이야기, 생텍쥐페리의 『어린 왕자』가 문학 평론가...",beautiful story little prince love world adult...,life monotonousi chase chickens people chase m...,beautiful story world love little prince saint...


In [25]:
total_text = book.loc[0, '책소개_trans'] + book.loc[0, '책속으로_trans'] + book.loc[0, '서평_trans']
total_text

'beautiful story little prince love world adult contradict pure gaze young prince another starhwang hyun san solidified position literature translation translation french texture french text accurate beautiful sentiment sharp literary insightshe translate work try bring sincere style saint tech juperi prepare korean crystal version faithfully utilizes value nuclear power plant work remember novel full sentence power move mindthe translator hwang hyun san interpret source power sincere passionate style author saint tech zuperiyou enjoy work new complete translation melt deep reflection life beautiful metaphor give deep afterglowlife monotonousi chase chickens people chase methe chicken everyone thatso im boredbut tame life bright sunlightunlike footstep hear sound footprint sound differentlythe footprint hide groundyour footstep call cave like musicand see wheat fieldsi dont eat breadwheat use meso sadbut hair goldenso tame something really happenwheat golden remind youso love wind whea

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

df = pd.read_csv('tweet_data_agumentation.csv', index_col = 0)

tfidf_vect_emo = TfidfVectorizer()
tfidf_vect_emo.fit_transform(df["content"])

model = joblib.load('SVM.pkl')
total_text2 = tfidf_vect_emo.transform(pd.Series(total_text))
model.predict_proba(total_text2)
sentiment = pd.DataFrame(model.predict_proba(total_text2),index=['prob']).T
sentiment['감정'] = ['empty','sadness','enthusiasm','worry','love','fun','hate','happiness','boredom','relief','anger']
sentiment2 = sentiment.sort_values(by='prob',ascending=False)
sentiment2
# sentiment 이용!

,prob,감정
3,0.387811,worry
1,0.232665,sadness
2,0.098190,enthusiasm
7,0.097348,happiness
4,0.085197,love
5,0.044240,fun
9,0.037261,relief
6,0.011235,hate
0,0.004563,empty
8,0.000838,boredom


# audio feature랑 text 감정

In [27]:
audio_data = data.iloc[:,-12:-1]
audio_data

,empty_a,sadness_a,enthusiasm_a,worry_a,love_a,fun_a,hate_a,happiness_a,boredom_a,relief_a,anger_a
0,0.000225,0.000191,0.000152,1.000000,0.000023,0.000135,1.214844e-05,0.000221,0.000104,0.000052,1.310354e-05
1,0.000062,0.000079,0.001033,0.000177,0.000280,0.001177,2.478030e-04,0.001157,0.999988,0.000124,2.485695e-04
2,0.000284,0.000221,0.000088,0.000382,0.000073,0.000093,1.870017e-05,0.000099,0.000147,1.000000,1.633500e-05
3,0.000252,0.000261,0.000068,0.000342,0.000151,0.000067,1.677677e-05,0.000076,0.000135,0.999990,1.130652e-05
4,0.000165,0.000172,0.000190,0.000034,0.000072,0.000203,1.000000e+00,0.000166,0.000486,0.000042,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...
8515,1.000000,1.000000,0.000048,0.000154,1.000000,0.000033,4.020569e-05,0.000032,0.000517,0.000025,5.175872e-05
8516,0.015458,0.015540,0.000100,0.000243,0.016848,0.000087,9.876737e-01,0.000068,0.000284,0.000351,9.881506e-01
8517,1.000000,1.000000,0.000298,0.000185,1.000000,0.000290,1.177873e-04,0.000280,0.000078,0.000063,1.206514e-04
8518,1.000000,1.000000,0.000120,0.000130,1.000000,0.000105,1.357035e-04,0.000088,0.000229,0.000052,1.641192e-04


In [28]:
sentiment

,prob,감정
0,0.004563,empty
1,0.232665,sadness
2,0.098190,enthusiasm
3,0.387811,worry
4,0.085197,love
5,0.044240,fun
6,0.011235,hate
7,0.097348,happiness
8,0.000838,boredom
9,0.037261,relief


In [29]:
sentiment_prob = sentiment['prob']
sentiment_prob.index = sentiment['감정']
sentiment_prob

감정
empty         0.004563
sadness       0.232665
enthusiasm    0.098190
worry         0.387811
love          0.085197
fun           0.044240
hate          0.011235
happiness     0.097348
boredom       0.000838
relief        0.037261
anger         0.000653
Name: prob, dtype: float64

In [30]:
audio_data.columns = ['empty', 'sadness', 'enthusiasm', 'worry', 'love', 'fun', 'hate',
       'happiness', 'boredom', 'relief', 'anger']

In [31]:
audio_data_1 = pd.concat([sentiment_prob,audio_data.T],axis=1).T
audio_data_1

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
prob,0.004563,0.232665,0.098190,0.387811,0.085197,0.044240,1.123480e-02,0.097348,0.000838,0.037261,6.529049e-04
0,0.000225,0.000191,0.000152,1.000000,0.000023,0.000135,1.214844e-05,0.000221,0.000104,0.000052,1.310354e-05
1,0.000062,0.000079,0.001033,0.000177,0.000280,0.001177,2.478030e-04,0.001157,0.999988,0.000124,2.485695e-04
2,0.000284,0.000221,0.000088,0.000382,0.000073,0.000093,1.870017e-05,0.000099,0.000147,1.000000,1.633500e-05
3,0.000252,0.000261,0.000068,0.000342,0.000151,0.000067,1.677677e-05,0.000076,0.000135,0.999990,1.130652e-05
...,...,...,...,...,...,...,...,...,...,...,...
8515,1.000000,1.000000,0.000048,0.000154,1.000000,0.000033,4.020569e-05,0.000032,0.000517,0.000025,5.175872e-05
8516,0.015458,0.015540,0.000100,0.000243,0.016848,0.000087,9.876737e-01,0.000068,0.000284,0.000351,9.881506e-01
8517,1.000000,1.000000,0.000298,0.000185,1.000000,0.000290,1.177873e-04,0.000280,0.000078,0.000063,1.206514e-04
8518,1.000000,1.000000,0.000120,0.000130,1.000000,0.000105,1.357035e-04,0.000088,0.000229,0.000052,1.641192e-04


In [32]:
col = ['book']+list(data['name'])

In [33]:
cosine_sim_audio = cosine_similarity(audio_data_1)
cosine_sim_audio_df = pd.DataFrame(cosine_sim_audio, index = col, columns=col)
cosine_sim_audio_df

,book,Still With You,death bed (coffee for your head),Nan-Nan,Mo-Eh-Wa,YASASHISA,Kiri Ga Naikara,Tsumi No Kaori,Cho Si Noccha Te,Tokuninai,Shinunoga E-Wa,Kazeyo,SAYONARA Baby,Kaerou,My Head & My Heart,H.E.A.V.E.N,Kings & Queens,Naked,Tattoo,OMG What's Happening,Call Me Tonight,Born to the Night,Torn,Take You To Hell,Who's Laughing Now,Belladonna,Rumors,So Am I,Salt,Sweet but Psycho,Stuck with U (with Justin Bieber),DÁKITI,Ashley,clementine,Graveyard,You should be sad,Forever ... (is a long time),Dominic’s Interlude,I HATE EVERYBODY,3am,...,I Found You,I Don't Wanna Go To Heaven,Tú Me Vuelves Loco,No Puedo Más,Duele El Amor,Tu Amor Por Siempre,Cuando Te Vi,Sólo Quédate En Silencio,Como La Flor - Remastered,Mujer Latina,Mi Buen Amor,Desnúdate Mujer,Ricota,Sueño De Ti,Me Haces Falta,Back to ZEROBASE,2:30,2013 Skit,Riflessi rossi (feat. Guè & Luchè),Un milione di volte (feat. Sfera Ebbasta),Happier Than Ever - Edit,Rescue Me,Better,Do It To It,Drugs From Amsterdam,I Need A Painkiller,Oh Lord,Seek & Hide,You Know You Like It - Tchami Remix,Impossible - Coco & Breezy Remix,My Heart Goes (La Di Da),Back To You,Tick Tack - Extended Mix,Turn Me Up - ViP Mix,Miserable Man,Murder in the City,September Song - Guitar Acoustic,Control,Heart's Content - Strings Mix,We Don't Have To Take Our Clothes Off
book,1.000000,0.801205,0.002526,0.077436,0.077425,0.017517,0.335807,0.335865,0.335891,0.077163,0.338491,0.002087,0.077069,0.077262,0.335863,0.335907,0.801317,0.801297,0.077427,0.335802,0.801317,0.077106,0.801344,0.801224,0.077224,0.801231,0.017539,0.077252,0.077523,0.017585,0.077160,0.075035,0.077596,0.002036,0.335877,0.017543,0.001889,0.384223,0.081894,0.801336,...,0.017520,0.384588,0.077231,0.335826,0.336207,0.384790,0.017718,0.384790,0.077021,0.335888,0.384661,0.077286,0.335883,0.019208,0.077436,0.017512,0.077128,0.384994,0.077335,0.338640,0.017509,0.335688,0.801239,0.335869,0.077299,0.335862,0.803697,0.338899,0.806250,0.077196,0.336068,0.801309,0.335815,0.335805,0.384544,0.384579,0.025115,0.384667,0.384588,0.007076
Still With You,0.801205,1.000000,0.000282,0.000434,0.000394,0.000042,0.000338,0.000342,0.000414,0.000234,0.003580,0.000195,0.000076,0.000325,0.000364,0.000405,1.000000,1.000000,0.000477,0.000335,1.000000,0.000134,1.000000,0.999999,0.000228,1.000000,0.000028,0.000223,0.000622,0.000041,0.000200,0.090602,0.000694,0.000262,0.000521,0.000051,0.000206,0.000388,0.006011,0.999965,...,0.000042,0.000337,0.000227,0.000362,0.000644,0.000361,0.000144,0.000301,0.000075,0.000351,0.000384,0.000353,0.000351,0.000350,0.000494,0.000036,0.000112,0.000366,0.000238,0.003540,0.000031,0.000306,1.000000,0.000359,0.000093,0.000339,0.999971,0.003851,0.999877,0.000197,0.000370,1.000000,0.000394,0.000394,0.000288,0.000343,0.000197,0.000360,0.000328,0.000130
death bed (coffee for your head),0.002526,0.000282,1.000000,0.000271,0.000259,0.000695,0.001881,0.001895,0.001914,0.000370,0.001921,0.999999,0.000270,0.000257,0.001908,0.001969,0.000206,0.000248,0.000385,0.001907,0.000235,0.000307,0.000225,0.000213,0.000470,0.000224,0.000730,0.000401,0.000363,0.000703,0.000597,0.995912,0.000367,0.999999,0.001943,0.000622,0.999998,0.002470,0.000166,0.000230,...,0.000699,0.000561,0.000413,0.001902,0.002008,0.000699,0.000891,0.002409,0.000520,0.001916,0.000315,0.000277,0.001872,0.000483,0.000172,0.000940,0.000278,0.000523,0.000239,0.002362,0.000637,0.001876,0.000206,0.001887,0.000164,0.001934,0.000269,0.003130,0.000250,0.000236,0.001923,0.000251,0.001879,0.001973,0.000616,0.000542,0.000559,0.000289,0.000376,0.999880
Nan-Nan,0.077436,0.000434,0.000271,1.000000,1.000000,0.000054,0.000191,0.000182,0.000212,1.000000,0.000354,0.000238,1.000000,1.000000,0.000179,0.000203,0.000406,0.000506,1.000000,0.000181,0.000500,1.000000,0.000442,0.000397,1.000000,0.000408,0.000042,1.000000,1.000000,0.000037,1.000000,0.000705,1.000000,0.000194,0.000182,0.000075,0.000186,0.036065,0.999976,0.000386,...,0.000043,0.000352,1.000000,0.000185,0.000288,0.002079,0.000033,0.000802,1.000000,0.000181,0.00

In [34]:
audio_sim = cosine_sim_audio_df['book']
audio_sim

book                                     1.000000
Still With You                           0.801205
death bed (coffee for your head)         0.002526
Nan-Nan                                  0.077436
Mo-Eh-Wa                                 0.077425
                                           ...   
Murder in the City                       0.384579
September Song - Guitar Acoustic         0.025115
Control                                  0.384667
Heart's Content - Strings Mix            0.384588
We Don't Have To Take Our Clothes Off    0.007076
Name: book, Length: 8521, dtype: float64

# 가사랑 text

In [35]:
lyrics_data = data.iloc[:,5:-12]
lyrics_data

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,0.016461,0.265974,0.067839,0.144616,0.116033,0.206217,0.015079,0.134595,0.002582,0.028249,0.002356
1,0.122187,0.059084,0.039108,0.113228,0.024429,0.023013,0.015683,0.212476,0.004940,0.384384,0.001468
2,0.005816,0.248961,0.003242,0.483580,0.044740,0.008447,0.022421,0.152159,0.000405,0.029745,0.000483
3,0.077518,0.249271,0.011364,0.406069,0.011648,0.038887,0.005062,0.123563,0.002235,0.072786,0.001598
4,0.001735,0.235752,0.001410,0.463885,0.142529,0.009176,0.010140,0.120540,0.000858,0.013238,0.000737
...,...,...,...,...,...,...,...,...,...,...,...
8515,0.011245,0.162807,0.006966,0.456847,0.082303,0.040452,0.002504,0.231635,0.000722,0.004143,0.000376
8516,0.021068,0.348507,0.035223,0.432478,0.046740,0.014127,0.007841,0.074129,0.001142,0.017115,0.001629
8517,0.002924,0.170607,0.004646,0.211214,0.003779,0.007508,0.575163,0.020068,0.000557,0.002283,0.001252
8518,0.004582,0.297325,0.004344,0.583962,0.050943,0.004454,0.001543,0.051992,0.000209,0.000479,0.000167


In [36]:
lyrics_data.columns

Index(['empty', 'sadness', 'enthusiasm', 'worry', 'love', 'fun', 'hate',
       'happiness', 'boredom', 'relief', 'anger'],
      dtype='object')

In [37]:
lyrics_data_1 = pd.concat([sentiment_prob,lyrics_data.T],axis=1).T
lyrics_data_1 

,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
prob,0.004563,0.232665,0.098190,0.387811,0.085197,0.044240,0.011235,0.097348,0.000838,0.037261,0.000653
0,0.016461,0.265974,0.067839,0.144616,0.116033,0.206217,0.015079,0.134595,0.002582,0.028249,0.002356
1,0.122187,0.059084,0.039108,0.113228,0.024429,0.023013,0.015683,0.212476,0.004940,0.384384,0.001468
2,0.005816,0.248961,0.003242,0.483580,0.044740,0.008447,0.022421,0.152159,0.000405,0.029745,0.000483
3,0.077518,0.249271,0.011364,0.406069,0.011648,0.038887,0.005062,0.123563,0.002235,0.072786,0.001598
...,...,...,...,...,...,...,...,...,...,...,...
8515,0.011245,0.162807,0.006966,0.456847,0.082303,0.040452,0.002504,0.231635,0.000722,0.004143,0.000376
8516,0.021068,0.348507,0.035223,0.432478,0.046740,0.014127,0.007841,0.074129,0.001142,0.017115,0.001629
8517,0.002924,0.170607,0.004646,0.211214,0.003779,0.007508,0.575163,0.020068,0.000557,0.002283,0.001252
8518,0.004582,0.297325,0.004344,0.583962,0.050943,0.004454,0.001543,0.051992,0.000209,0.000479,0.000167


In [38]:
col = ['book']+list(data['name'])

In [39]:
cosine_sim_lyrics = cosine_similarity(lyrics_data_1)
cosine_sim_lyrics_df = pd.DataFrame(cosine_sim_lyrics, index =col, columns=col)
cosine_sim_lyrics_df

,book,Still With You,death bed (coffee for your head),Nan-Nan,Mo-Eh-Wa,YASASHISA,Kiri Ga Naikara,Tsumi No Kaori,Cho Si Noccha Te,Tokuninai,Shinunoga E-Wa,Kazeyo,SAYONARA Baby,Kaerou,My Head & My Heart,H.E.A.V.E.N,Kings & Queens,Naked,Tattoo,OMG What's Happening,Call Me Tonight,Born to the Night,Torn,Take You To Hell,Who's Laughing Now,Belladonna,Rumors,So Am I,Salt,Sweet but Psycho,Stuck with U (with Justin Bieber),DÁKITI,Ashley,clementine,Graveyard,You should be sad,Forever ... (is a long time),Dominic’s Interlude,I HATE EVERYBODY,3am,...,I Found You,I Don't Wanna Go To Heaven,Tú Me Vuelves Loco,No Puedo Más,Duele El Amor,Tu Amor Por Siempre,Cuando Te Vi,Sólo Quédate En Silencio,Como La Flor - Remastered,Mujer Latina,Mi Buen Amor,Desnúdate Mujer,Ricota,Sueño De Ti,Me Haces Falta,Back to ZEROBASE,2:30,2013 Skit,Riflessi rossi (feat. Guè & Luchè),Un milione di volte (feat. Sfera Ebbasta),Happier Than Ever - Edit,Rescue Me,Better,Do It To It,Drugs From Amsterdam,I Need A Painkiller,Oh Lord,Seek & Hide,You Know You Like It - Tchami Remix,Impossible - Coco & Breezy Remix,My Heart Goes (La Di Da),Back To You,Tick Tack - Extended Mix,Turn Me Up - ViP Mix,Miserable Man,Murder in the City,September Song - Guitar Acoustic,Control,Heart's Content - Strings Mix,We Don't Have To Take Our Clothes Off
book,1.000000,0.787840,0.434940,0.968089,0.958215,0.970050,0.977434,0.934927,0.973198,0.975436,0.957356,0.929191,0.902895,0.972439,0.876554,0.936418,0.948601,0.969589,0.742124,0.948103,0.860154,0.610711,0.968252,0.931391,0.950482,0.946403,0.929574,0.418394,0.956465,0.885718,0.910184,0.952034,0.473968,0.955558,0.959620,0.977834,0.970971,0.915446,0.449192,0.946766,...,0.314797,0.919054,0.936075,0.176630,0.800906,0.961733,0.935938,0.677092,0.181024,0.890422,0.952035,0.926390,0.930759,0.677620,0.681110,0.931721,0.973192,0.255415,0.224526,0.971178,0.966549,0.936790,0.771384,0.965597,0.263163,0.898352,0.896990,0.852074,0.956124,0.352654,0.892648,0.594580,0.469537,0.723996,0.947830,0.936591,0.969668,0.425899,0.957680,0.946887
Still With You,0.787840,1.000000,0.425674,0.701512,0.741244,0.720355,0.736091,0.579968,0.741391,0.806184,0.689724,0.567021,0.802509,0.724664,0.808830,0.815044,0.667349,0.701578,0.780694,0.631143,0.677608,0.663527,0.678403,0.784579,0.677751,0.844243,0.608177,0.445222,0.634785,0.808447,0.696185,0.623391,0.443117,0.654642,0.647486,0.718420,0.770616,0.676609,0.368811,0.712348,...,0.391319,0.550419,0.654406,0.280609,0.770668,0.663150,0.712341,0.654889,0.283915,0.803671,0.680471,0.724960,0.568145,0.729734,0.603688,0.833433,0.774393,0.222072,0.338613,0.728406,0.782200,0.700278,0.542465,0.797657,0.369668,0.675530,0.829771,0.829143,0.812240,0.426933,0.770901,0.681153,0.452202,0.860433,0.645661,0.704713,0.756607,0.340225,0.650681,0.805498
death bed (coffee for your head),0.434940,0.425674,1.000000,0.427589,0.523732,0.384234,0.418146,0.322419,0.463423,0.454470,0.312204,0.292101,0.306262,0.503198,0.423082,0.465277,0.453017,0.351191,0.389657,0.395259,0.374461,0.558275,0.387599,0.486150,0.341369,0.406576,0.377391,0.189595,0.327044,0.627742,0.313031,0.383082,0.220817,0.348515,0.325361,0.395475,0.398391,0.392694,0.168786,0.396043,...,0.131026,0.270854,0.271377,0.051773,0.287983,0.394905,0.517838,0.342431,0.055070,0.307129,0.402917,0.272015,0.305800,0.506790,0.568445,0.557239,0.441943,0.116926,0.119416,0.376437,0.328069,0.475215,0.408084,0.403388,0.126925,0.485258,0.402112,0.314220,0.464111,0.153121,0.289112,0.265163,0.148591,0.486258,0.286708,0.449359,0.362375,0.161462,0.308787,0.410427
Nan-Nan,0.968089,0.701512,0.427589,1.000000,0.978559,0.981930,0.994108,0.970731,0.981328,0.981550,0.963880,0.960619,0.887154,0.969337,0.844613,0.951714,0.984507,0.986574,0.632195,0.905007,0.842306,0.631381,0.983702,0.904773,0.969886,0.931200,0.959960,0.341756,0.978968,0.899713,0.924058,0.985043,0.487501,0.956407,0.979618,0.996525,0.974808,0.902867,0.465257,0.959056,...,0.221078,0.951891,0.953757,0.079450,0.739024,0.979528,0.954632,0.590944,0.084257,0.852662,0.96

In [40]:
lyrics_sim = cosine_sim_lyrics_df['book']
lyrics_sim

book                                     1.000000
Still With You                           0.787840
death bed (coffee for your head)         0.434940
Nan-Nan                                  0.968089
Mo-Eh-Wa                                 0.958215
                                           ...   
Murder in the City                       0.936591
September Song - Guitar Acoustic         0.969668
Control                                  0.425899
Heart's Content - Strings Mix            0.957680
We Don't Have To Take Our Clothes Off    0.946887
Name: book, Length: 8521, dtype: float64

# 키워드랑 text 유사도

In [41]:
keyword_data = data['key_word']
keyword_data

0       face want see eye tell ecstatic memory even da...
1       stay awake long go bed make cup coffee head ge...
2       say didnt ask anything face w swear corner flo...
3       night blown cold wind dont wander blow inner w...
4       dream dream shade person kill kindness dream s...
                              ...                        
8515    get murder city revenge name one person dead p...
8516    innocent darling use talk til morning mixtape ...
8517    wan na lose control even though old know shake...
8518    love live without u one thing know sure love f...
8519    take clothes good time oh oh could dance party...
Name: key_word, Length: 8520, dtype: object

In [42]:
total_text

'beautiful story little prince love world adult contradict pure gaze young prince another starhwang hyun san solidified position literature translation translation french texture french text accurate beautiful sentiment sharp literary insightshe translate work try bring sincere style saint tech juperi prepare korean crystal version faithfully utilizes value nuclear power plant work remember novel full sentence power move mindthe translator hwang hyun san interpret source power sincere passionate style author saint tech zuperiyou enjoy work new complete translation melt deep reflection life beautiful metaphor give deep afterglowlife monotonousi chase chickens people chase methe chicken everyone thatso im boredbut tame life bright sunlightunlike footstep hear sound footprint sound differentlythe footprint hide groundyour footstep call cave like musicand see wheat fieldsi dont eat breadwheat use meso sadbut hair goldenso tame something really happenwheat golden remind youso love wind whea

In [43]:
book_song_cont1 = pd.DataFrame({"text": total_text}, index = range(1))
book_song_cont1

,text
0,beautiful story little prince love world adult...


In [44]:
book_song_cont2 = pd.DataFrame({"text": keyword_data})
book_song_cont2

,text
0,face want see eye tell ecstatic memory even da...
1,stay awake long go bed make cup coffee head ge...
2,say didnt ask anything face w swear corner flo...
3,night blown cold wind dont wander blow inner w...
4,dream dream shade person kill kindness dream s...
...,...
8515,get murder city revenge name one person dead p...
8516,innocent darling use talk til morning mixtape ...
8517,wan na lose control even though old know shake...
8518,love live without u one thing know sure love f...


In [45]:
keyword_data_1 = pd.concat([book_song_cont1, book_song_cont2], axis=0).reset_index(drop=True)
keyword_data_1

,text
0,beautiful story little prince love world adult...
1,face want see eye tell ecstatic memory even da...
2,stay awake long go bed make cup coffee head ge...
3,say didnt ask anything face w swear corner flo...
4,night blown cold wind dont wander blow inner w...
...,...
8516,get murder city revenge name one person dead p...
8517,innocent darling use talk til morning mixtape ...
8518,wan na lose control even though old know shake...
8519,love live without u one thing know sure love f...


In [46]:
tfidf_vect_cont = TfidfVectorizer()
tfidf_matrix_cont = tfidf_vect_cont.fit_transform(keyword_data_1['text'])
tfidf_array_cont = tfidf_matrix_cont.toarray()
tfidf_df_cont = pd.DataFrame(tfidf_array_cont, columns=tfidf_vect_cont.get_feature_names_out())
tfidf_df_cont

,00,000,00s,01,03,04,050,06,07,085,09,10,100,1000,1004,100k,100th,103,107,10k,11,117,12,120,12th,13,132,14,14th,15,150,1500s,15th,16,160,1600,16yrold,17,18,180,...,zhl,zig,zigzag,zil,zillow,zimbabwe,zindagi,zinnia,zion,zip,zipette,zl1,zobo,zombie,zombified,zone,zonin,zoo,zoom,zoomers,zoomin,zori,zouker,zozi,zozibini,zu,zuckerberg,zum,zumba,zumbe,zun,zunk,zuperi,zuperiit,zuperis,zuperiyou,zurna,zyaada,zyla,zzoil
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017559,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05689,0.018963,0.018963,0.018963,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8516,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
8517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
8518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
8519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [47]:
col = ['book']+list(data['name'])

In [48]:
cosine_sim_keyword = cosine_similarity(tfidf_array_cont)
cosine_sim_keyword_df = pd.DataFrame(cosine_sim_keyword, index = col, columns=col)
cosine_sim_keyword_df

,book,Still With You,death bed (coffee for your head),Nan-Nan,Mo-Eh-Wa,YASASHISA,Kiri Ga Naikara,Tsumi No Kaori,Cho Si Noccha Te,Tokuninai,Shinunoga E-Wa,Kazeyo,SAYONARA Baby,Kaerou,My Head & My Heart,H.E.A.V.E.N,Kings & Queens,Naked,Tattoo,OMG What's Happening,Call Me Tonight,Born to the Night,Torn,Take You To Hell,Who's Laughing Now,Belladonna,Rumors,So Am I,Salt,Sweet but Psycho,Stuck with U (with Justin Bieber),DÁKITI,Ashley,clementine,Graveyard,You should be sad,Forever ... (is a long time),Dominic’s Interlude,I HATE EVERYBODY,3am,...,I Found You,I Don't Wanna Go To Heaven,Tú Me Vuelves Loco,No Puedo Más,Duele El Amor,Tu Amor Por Siempre,Cuando Te Vi,Sólo Quédate En Silencio,Como La Flor - Remastered,Mujer Latina,Mi Buen Amor,Desnúdate Mujer,Ricota,Sueño De Ti,Me Haces Falta,Back to ZEROBASE,2:30,2013 Skit,Riflessi rossi (feat. Guè & Luchè),Un milione di volte (feat. Sfera Ebbasta),Happier Than Ever - Edit,Rescue Me,Better,Do It To It,Drugs From Amsterdam,I Need A Painkiller,Oh Lord,Seek & Hide,You Know You Like It - Tchami Remix,Impossible - Coco & Breezy Remix,My Heart Goes (La Di Da),Back To You,Tick Tack - Extended Mix,Turn Me Up - ViP Mix,Miserable Man,Murder in the City,September Song - Guitar Acoustic,Control,Heart's Content - Strings Mix,We Don't Have To Take Our Clothes Off
book,1.000000,0.042324,0.011846,0.041878,0.039410,0.029523,0.004307,0.036359,0.026000,0.019037,0.023132,0.026868,0.015252,0.066543,0.022675,0.009216,0.042657,0.034108,0.000000,0.034301,0.012350,0.008820,0.010261,0.016712,0.023681,0.035707,0.001738,0.013330,0.013444,0.024697,0.006449,0.019097,0.002212,0.060469,0.006249,0.006099,0.014401,0.081402,0.025101,0.010753,...,0.019203,0.007400,0.018452,0.008288,0.017142,0.029722,0.030298,0.024137,0.041933,0.009366,0.043868,0.030640,0.012224,0.032714,0.015669,0.006480,0.022285,0.021571,0.034768,0.031791,0.021636,0.009919,0.028911,0.003861,0.001080,0.006389,0.010354,0.034817,0.025096,0.030302,0.010787,0.017936,0.013772,0.010741,0.001832,0.035823,0.005258,0.024125,0.014471,0.015990
Still With You,0.042324,1.000000,0.019364,0.064304,0.069099,0.005156,0.012316,0.031760,0.060666,0.032768,0.019600,0.044828,0.009410,0.103579,0.033255,0.027956,0.059167,0.048903,0.000000,0.046437,0.002473,0.072301,0.033729,0.014279,0.054813,0.021936,0.069518,0.023157,0.056035,0.036022,0.043382,0.013101,0.000000,0.021353,0.018957,0.000000,0.000000,0.083246,0.059912,0.008134,...,0.012819,0.000000,0.025532,0.045268,0.078932,0.027738,0.021806,0.105566,0.017824,0.033146,0.073462,0.049380,0.030454,0.063828,0.065191,0.009578,0.017503,0.011649,0.024802,0.039984,0.000000,0.062907,0.000000,0.002385,0.111370,0.000000,0.018182,0.111171,0.066935,0.078070,0.019701,0.000000,0.012988,0.015885,0.012615,0.042763,0.004411,0.063795,0.006678,0.058055
death bed (coffee for your head),0.011846,0.019364,1.000000,0.000000,0.022845,0.000000,0.000000,0.000000,0.000000,0.014346,0.000000,0.042203,0.000000,0.042513,0.086789,0.000000,0.023188,0.008571,0.000000,0.006145,0.029300,0.000000,0.044672,0.009907,0.076436,0.012651,0.006989,0.007244,0.006562,0.000000,0.026370,0.021490,0.000000,0.000000,0.000000,0.007188,0.051773,0.007135,0.038865,0.004929,...,0.039987,0.029048,0.000000,0.059269,0.027634,0.000000,0.033988,0.129628,0.013440,0.000000,0.007696,0.013535,0.000000,0.000000,0.000000,0.016579,0.042058,0.000000,0.000000,0.000000,0.021771,0.000000,0.000000,0.000000,0.000000,0.021379,0.051520,0.113578,0.000000,0.014613,0.015387,0.096907,0.000000,0.006336,0.000000,0.064744,0.000000,0.010995,0.016883,0.007835
Nan-Nan,0.041878,0.064304,0.000000,1.000000,0.017875,0.060499,0.039351,0.016875,0.020848,0.044422,0.007834,0.022014,0.015556,0.088802,0.010640,0.000000,0.031662,0.024707,0.000000,0.049216,0.007106,0.009167,0.062946,0.030848,0.047881,0.031461,0.004337,0.006045,0.020191,0.004615,0.104699,0.015444,0.000000,0.000000,0.020815,0.021056,0.032745,0.030539,0.000000,0.007691,...,0.066848,0.000000,0.003608,0.035454,0.027411,0.000000,0.044788,0.035623,0.055737,0.000000,0.03

In [49]:
keyword_sim = cosine_sim_keyword_df['book']
keyword_sim

book                                     1.000000
Still With You                           0.042324
death bed (coffee for your head)         0.011846
Nan-Nan                                  0.041878
Mo-Eh-Wa                                 0.039410
                                           ...   
Murder in the City                       0.035823
September Song - Guitar Acoustic         0.005258
Control                                  0.024125
Heart's Content - Strings Mix            0.014471
We Don't Have To Take Our Clothes Off    0.015990
Name: book, Length: 8521, dtype: float64

# 전체 유사도 계산

In [50]:
total_sim  = 0.8*audio_sim + 0.1*lyrics_sim + 0.1*keyword_sim

recommend_song = total_sim.sort_values(ascending=False)[1:6].index
total_sim_df = pd.DataFrame(total_sim[1:])
total_sim_df = total_sim_df.reset_index()
total_sim_df.columns = ['name','book']

top_five = total_sim_df.sort_values(by='book',ascending=False)[:5]
index = total_sim_df.sort_values(by='book',ascending=False)[:5].index.sort_values()

In [51]:
artist = data.iloc[index][['url','name','Artist']]
top_five_df = pd.merge(artist,top_five,on='name').sort_values(by='book',ascending=False).drop_duplicates()
top_five_df 

,url,name,Artist,book
0,https://open.spotify.com/track/6wQlQrTY5mVS8EG...,Smile (with The Weeknd),Juice WRLD,0.794459
3,https://open.spotify.com/track/5MWXOo8DJwgODtP...,cinderella's dead,EMELINE,0.792672
4,https://open.spotify.com/track/5uSERYH51rYFi2S...,WORLD ON FIRE,Lancey Foux,0.791234
1,https://open.spotify.com/track/7Bk0uXKk1uPT0Xu...,Fiel,Los Legendarios,0.789551
2,https://open.spotify.com/track/3r8RuvgbX9s7amm...,Ditto,NewJeans,0.788209


In [52]:
total_sim  = 0*audio_sim + 0.5*lyrics_sim + 0.5*keyword_sim

recommend_song = total_sim.sort_values(ascending=False)[1:6].index
total_sim_df_1 = pd.DataFrame(total_sim[1:])
total_sim_df_1 = total_sim_df_1.reset_index()
total_sim_df_1.columns = ['name','book']

top_five_1 = total_sim_df_1.sort_values(by='book',ascending=False)[:5]
index_1 = total_sim_df_1.sort_values(by='book',ascending=False)[:5].index.sort_values()

In [53]:
artist = data.iloc[index_1][['url','name','Artist']]
top_five_df_1 = pd.merge(artist,top_five_1,on='name').sort_values(by='book',ascending=False).drop_duplicates()
top_five_df_1 

,url,name,Artist,book
4,https://open.spotify.com/track/4NbrfpIYdwG9S6f...,Petits princes,Mentissa,0.591380
2,https://open.spotify.com/track/6N9U1wkWniSXD4T...,WASTING TIME (FEAT. DRAKE & THE NEPTUNES),Brent Faiyaz,0.548790
3,https://open.spotify.com/track/3F8ydJz1se1vIxZ...,PRINCE OF DARKNESS,SHADXWBXRN,0.544303
0,https://open.spotify.com/track/5TUZhrb8UaaA76N...,Lost in Translation,The Neighbourhood,0.542612
1,https://open.spotify.com/track/1VMBapbhHidf6AL...,Lonely,Imagine Dragons,0.541962


In [54]:
final_df = pd.concat([top_five_df,top_five_df_1])
final_df = final_df.sort_values(by='book',ascending=False)

In [55]:
url_list = top_five_df['url']
url_list

0    https://open.spotify.com/track/6wQlQrTY5mVS8EG...
3    https://open.spotify.com/track/5MWXOo8DJwgODtP...
4    https://open.spotify.com/track/5uSERYH51rYFi2S...
1    https://open.spotify.com/track/7Bk0uXKk1uPT0Xu...
2    https://open.spotify.com/track/3r8RuvgbX9s7amm...
Name: url, dtype: object

In [56]:
top_five_df.iloc[1]

url       https://open.spotify.com/track/5MWXOo8DJwgODtP...
name                                      cinderella's dead
Artist                                              EMELINE
book                                               0.792672
Name: 3, dtype: object

In [57]:
print('TOP 1\n')
print('제목 : {0} \n가수 : {1} \nurl : {2} \n유사도 : {3:.4f}'.format(top_five_df.iloc[1]['name'],top_five_df.iloc[1]['Artist'],top_five_df.iloc[1]['url'],top_five_df.iloc[1]['book']))

TOP 1

제목 : cinderella's dead 
가수 : EMELINE 
url : https://open.spotify.com/track/5MWXOo8DJwgODtPGaietNz 
유사도 : 0.7927


In [58]:
top_five_df['url'][0]

'https://open.spotify.com/track/6wQlQrTY5mVS8EGaFZVwVF'

# 가사 데이터

In [89]:
lyrics_1 = pd.read_excel('/Users/seoeunseo/Desktop/deep.daiv/프로젝트/가사/lyrics1_summary.xlsx',index_col=0)
lyrics_1

,name,id,url,Artist,lyrics,lyrics_english,summary
0,Still With You,0eFMbKCRw8KByXyWBw8WO7,https://open.spotify.com/track/0eFMbKCRw8KByXy...,Jung Kook,날 스치는 그대의 옅은 그 목소리\n내 이름을 한 번만 더 불러주세요\n얼어버린 노...,Your pale voice\nPlease call my name once more...,<pad> the moment a day I would have captured m...
1,death bed (coffee for your head),7eJMfftS33KTjuF7lTsMCx,https://open.spotify.com/track/7eJMfftS33KTjuF...,Powfu,Don t stay awake for too long don t go to bed...,Don t stay awake for too long don t go to bed...,<pad> don t stay awake for too long don t go t...
2,Nan-Nan,2CbGuO0LtVvbh3umN3mDwM,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,Fujii Kaze,あんたのその歯に はさがった青さ粉に\nふれるべきか否かで少し悩んでる\n口にしない方がいい...,In the bluish powder that has exposed to your ...,<pad> what is it Before I said Then what Why d...
3,Mo-Eh-Wa,1JRNrJKTQhfvxII8kkIcAQ,https://open.spotify.com/track/1JRNrJKTQhfvxII...,Fujii Kaze,さぁ羽のばして ここから\n捉われてばっか だったから\n行き詰った悦び手放す時は今\n心軽...,Now from here\nBecause it was just caught\nNow...,<pad> the night was blown by the cold wind Don...
4,YASASHISA,57u1FaZlztApkcT5x5XNuD,https://open.spotify.com/track/57u1FaZlztApkcT...,Fujii Kaze,今何を見ていた あなたの夢を見た\n優しさに殺られた あの人の木陰で\n今何を見ていた あな...,What your dreams were now having your dreams\n...,<pad> what your dreams were now having your dr...
...,...,...,...,...,...,...,...
1747,Devil's Advocate,6H4agRAgmEuVfuuqIWgAT2,https://open.spotify.com/track/6H4agRAgmEuVfuu...,The Neighbourhood,Whoa\nOne two three four\nWhoa\nHaha\n♪\nOh...,Whoa\nOne two three four\nWhoa\nHaha\n♪\nOh...,<pad> i trade the whip out for a bike uh Desig...
1748,Hell or High Water,4G4aFRf3j194nyGUomu2KA,https://open.spotify.com/track/4G4aFRf3j194nyG...,The Neighbourhood,I went through hell to get to high water\nAnd ...,I went through hell to get to high water\nAnd ...,<pad> I tripped and fell in the wishing well I...
1749,Cherry Flavoured,3qyveovuSWvxXEMICuLlp3,https://open.spotify.com/track/3qyveovuSWvxXEM...,The Neighbourhood,Sweet and sour motivation\n♪\nWish I could kee...,Sweet and sour motivation\n♪\nWish I could kee...,<pad> I sold my soul A long time ago I feel li...
1750,The Mono-Tones,4sKIExrAk4vJY52XLgCsnG,https://open.spotify.com/track/4sKIExrAk4vJY52...,The Neighbourhood,Boys boys boys boys boys\nVoice in my head...,Boys boys boys boys boys\nVoice in my head...,<pad> Boys boys boys boys boys Voice in my hea...


In [90]:
lyrics_2 = pd.read_excel('/Users/seoeunseo/Desktop/deep.daiv/프로젝트/가사/lyrics2_summary.xlsx',index_col=0)
lyrics_3 = pd.read_excel('/Users/seoeunseo/Desktop/deep.daiv/프로젝트/가사/lyrics3_summary.xlsx',index_col=0)
lyrics_4 = pd.read_excel('/Users/seoeunseo/Desktop/deep.daiv/프로젝트/가사/lyrics4_summary.xlsx',index_col=0)
lyrics_5 = pd.read_excel('/Users/seoeunseo/Desktop/deep.daiv/프로젝트/가사/lyrics5_summary.xlsx',index_col=0)
lyrics_6 = pd.read_excel('/Users/seoeunseo/Desktop/deep.daiv/프로젝트/가사/lyrics6_summary.xlsx',index_col=0)
lyrics_7 = pd.read_excel('/Users/seoeunseo/Desktop/deep.daiv/프로젝트/가사/lyrics7_8_summary.xlsx',index_col=0)
lyrics_8 = pd.read_excel('/Users/seoeunseo/Desktop/deep.daiv/프로젝트/가사/lyrics9_10_summary.xlsx',index_col=0)

In [91]:
lyrics_7

,name,id,url,Artist,lyrics,lyrics_english,summary
0,Zeit,2KsHzM4L7ThhNJB6MHXkPx,https://open.spotify.com/track/2KsHzM4L7ThhNJB...,Rammstein,Manches sollte manches nicht\nWir sehen doch...,Some things shouldn't\nWe see we are blind\nWe...,<pad> the moment is over Time please stop Time...
1,Schwarz,2UK4qw712yzFh8vYYJGv2B,https://open.spotify.com/track/2UK4qw712yzFh8v...,Rammstein,Geh ich vor der Nacht zur Ruh\nDeck ich mich m...,I go to rest before the night\nI decorate myse...,<pad> the night holds their chest to many Drin...
2,Giftig,3Ye7vnZdrO2Gq5IyyH3cOp,https://open.spotify.com/track/3Ye7vnZdrO2Gq5I...,Rammstein,Im Wasser und zu Land\nIst uns viel Getier bek...,In the water and on land\nIs known to us a lot...,<pad> the poison slowly flows into my blood Oh...
3,Zick Zack,2w7Ca8DNYXBCgA6Bht7o0N,https://open.spotify.com/track/2w7Ca8DNYXBCgA6...,Rammstein,Schöner größer härter\nStraffer glatter st...,Nice bigger tougher\nTimer smoother stronger\n...,<pad> <unk> Ticktack ticket you are old Your t...
4,OK,1l0arEkM2grgUYMim5SggS,https://open.spotify.com/track/1l0arEkM2grgUYM...,Rammstein,Ohne Kondom\nOhne Kondom\nOhne Kondom\nOhne Ko...,Without condom\nWithout condom\nWithout condom...,<pad> without condom Without condom Without co...
...,...,...,...,...,...,...,...
3499,T-Shirt,27aZ1jJgnQTmJXILiRJZna,https://open.spotify.com/track/27aZ1jJgnQTmJXI...,Various Artists,Nard B\n♪\nTrenchWerk\nAy mama told me Ay ...,Nard B\n♪\nTrenchWerk\nAy mama told me Ay ...,<pad> white mama told me Ay not to sell work m...
3500,Ransom,6ImQLz5mLgGe3oxWdfby5h,https://open.spotify.com/track/6ImQLz5mLgGe3ox...,Various Artists,Turn you to a dancer\nYeah\n Internet Money b...,Turn you to a dancer\nYeah\n Internet Money b...,<pad> I got two twin opps leave em on a banner...
3501,Splashin,5TRnOafUNoWWGdHBJzsauR,https://open.spotify.com/track/5TRnOafUNoWWGdH...,Various Artists,I m havin dumb money\nAyy\nWe can fuck lil m...,I m havin dumb money\nAyy\nWe can fuck lil m...,<pad> ayy we can fuck lil mama I can t lip loc...
3502,It's A Vibe,1dmohp4TjFDo0UVsr8bmYH,https://open.spotify.com/track/1dmohp4TjFDo0UV...,Various Artists,Yeah hahaha\nYeah ooh\nYah\nThat s a vibe T...,Yeah hahaha\nYeah ooh\nYah\nThat s a vibe T...,<pad> if you get paid it s solely based on you...


In [92]:
lyrics = pd.concat([lyrics_1,lyrics_2,lyrics_3,lyrics_3,lyrics_4,lyrics_5,lyrics_6,lyrics_7.iloc[:,:-1],lyrics_8.iloc[:,:-1]],ignore_index=True)

In [93]:
lyrics 

,name,id,url,Artist,lyrics,lyrics_english,summary
0,Still With You,0eFMbKCRw8KByXyWBw8WO7,https://open.spotify.com/track/0eFMbKCRw8KByXy...,Jung Kook,날 스치는 그대의 옅은 그 목소리\n내 이름을 한 번만 더 불러주세요\n얼어버린 노...,Your pale voice\nPlease call my name once more...,<pad> the moment a day I would have captured m...
1,death bed (coffee for your head),7eJMfftS33KTjuF7lTsMCx,https://open.spotify.com/track/7eJMfftS33KTjuF...,Powfu,Don t stay awake for too long don t go to bed...,Don t stay awake for too long don t go to bed...,<pad> don t stay awake for too long don t go t...
2,Nan-Nan,2CbGuO0LtVvbh3umN3mDwM,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,Fujii Kaze,あんたのその歯に はさがった青さ粉に\nふれるべきか否かで少し悩んでる\n口にしない方がいい...,In the bluish powder that has exposed to your ...,<pad> what is it Before I said Then what Why d...
3,Mo-Eh-Wa,1JRNrJKTQhfvxII8kkIcAQ,https://open.spotify.com/track/1JRNrJKTQhfvxII...,Fujii Kaze,さぁ羽のばして ここから\n捉われてばっか だったから\n行き詰った悦び手放す時は今\n心軽...,Now from here\nBecause it was just caught\nNow...,<pad> the night was blown by the cold wind Don...
4,YASASHISA,57u1FaZlztApkcT5x5XNuD,https://open.spotify.com/track/57u1FaZlztApkcT...,Fujii Kaze,今何を見ていた あなたの夢を見た\n優しさに殺られた あの人の木陰で\n今何を見ていた あな...,What your dreams were now having your dreams\n...,<pad> what your dreams were now having your dr...
...,...,...,...,...,...,...,...
19270,September Song - Guitar Acoustic,4FhQ9BYFO3yFAASazZdvRX,https://open.spotify.com/track/4FhQ9BYFO3yFAAS...,Various Artists,Our love was stronger than lightning\nSoft as ...,Our love was stronger than lightning\nSoft as ...,NaN
19271,Control,70RiiOJdklpSmLSBnKlcYF,https://open.spotify.com/track/70RiiOJdklpSmLS...,Various Artists,Early in the morning I still get a little bit ...,Early in the morning I still get a little bit ...,NaN
19272,Heart's Content - Strings Mix,26Ow8vxj4n0uew1UP8CU56,https://open.spotify.com/track/26Ow8vxj4n0uew1...,Various Artists,Maybe you thought I hung the moon\nMaybe you t...,Maybe you thought I hung the moon\nMaybe you t...,NaN
19273,We Don't Have To Take Our Clothes Off,5JL1HLtR9jrlQraBCT1DL9,https://open.spotify.com/track/5JL1HLtR9jrlQra...,Various Artists,Not a word from your lips\nYou just took for ...,Not a word from your lips\nYou just took for ...,NaN


In [94]:
lyrics.to_excel('/Users/seoeunseo/Desktop/deep.daiv/프로젝트/가사/lyrics.xlsx')

In [83]:
lyrics_list = []

In [84]:
for i in top_five_df['url']:
    lyrics_list.append(lyrics[i== lyrics['url']]['lyrics'].values[0])

In [85]:
for i in top_five_df_1['url']:
    lyrics_list.append(lyrics[i== lyrics['url']]['lyrics'].values[0])

In [86]:
lyrics_list

['I d do anything in my power to see you just smile\nI want you to prosper and come proper\nEven if that means I ain t by your side\nI d do anything in my power to see you just smile\nI want you to prosper and come proper\nEven if that means I ain t by your side\nDevil on my shoulder tellin  me I ll die soon\nI don t really want that to impact you\nBut I don t know  maybe I m just paranoid\nI just want the best for you  I just want what s left of you\nOh  you tell me that you re sad inside\nI m sad that I can t satisfy\nYeah  I pray that I get it right this time\nMaybe we ll be alright\nOh  you tell me that you re sad inside\nI m sad that I can t satisfy\nYeah  I pray that I get it right this time\nMaybe we ll be alright\nI d do anything in my power to see you just smile\nI want you to prosper and come proper\nEven if that means I ain t by your side\nI d do anything in my power to see you just smile\nI want you to prosper and come proper\nEven if that means I ain t by your side\nOh  ye

In [87]:
pd.read_excel('/Users/seoeunseo/Desktop/deep.daiv/프로젝트/가사/lyrics.xlsx',index_col=0)

,name,id,url,Artist,lyrics,lyrics_english
0,Still With You,0eFMbKCRw8KByXyWBw8WO7,https://open.spotify.com/track/0eFMbKCRw8KByXy...,Jung Kook,날 스치는 그대의 옅은 그 목소리\n내 이름을 한 번만 더 불러주세요\n얼어버린 노...,Your pale voice\nPlease call my name once more...
1,death bed (coffee for your head),7eJMfftS33KTjuF7lTsMCx,https://open.spotify.com/track/7eJMfftS33KTjuF...,Powfu,Don t stay awake for too long don t go to bed...,Don t stay awake for too long don t go to bed...
2,Nan-Nan,2CbGuO0LtVvbh3umN3mDwM,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,Fujii Kaze,あんたのその歯に はさがった青さ粉に\nふれるべきか否かで少し悩んでる\n口にしない方がいい...,In the bluish powder that has exposed to your ...
3,Mo-Eh-Wa,1JRNrJKTQhfvxII8kkIcAQ,https://open.spotify.com/track/1JRNrJKTQhfvxII...,Fujii Kaze,さぁ羽のばして ここから\n捉われてばっか だったから\n行き詰った悦び手放す時は今\n心軽...,Now from here\nBecause it was just caught\nNow...
4,YASASHISA,57u1FaZlztApkcT5x5XNuD,https://open.spotify.com/track/57u1FaZlztApkcT...,Fujii Kaze,今何を見ていた あなたの夢を見た\n優しさに殺られた あの人の木陰で\n今何を見ていた あな...,What your dreams were now having your dreams\n...
...,...,...,...,...,...,...
19270,September Song - Guitar Acoustic,4FhQ9BYFO3yFAASazZdvRX,https://open.spotify.com/track/4FhQ9BYFO3yFAAS...,Various Artists,Our love was stronger than lightning\nSoft as ...,Our love was stronger than lightning\nSoft as ...
19271,Control,70RiiOJdklpSmLSBnKlcYF,https://open.spotify.com/track/70RiiOJdklpSmLS...,Various Artists,Early in the morning I still get a little bit ...,Early in the morning I still get a little bit ...
19272,Heart's Content - Strings Mix,26Ow8vxj4n0uew1UP8CU56,https://open.spotify.com/track/26Ow8vxj4n0uew1...,Various Artists,Maybe you thought I hung the moon\nMaybe you t...,Maybe you thought I hung the moon\nMaybe you t...
19273,We Don't Have To Take Our Clothes Off,5JL1HLtR9jrlQraBCT1DL9,https://open.spotify.com/track/5JL1HLtR9jrlQra...,Various Artists,Not a word from your lips\nYou just took for ...,Not a word from your lips\nYou just took for ...


In [99]:
lyrics[top_five_df['url'][0]== lyrics['url']]['lyrics_english'].values[0]

'I d do anything in my power to see you just smile\nI want you to prosper and come proper\nEven if that means I ain t by your side\nI d do anything in my power to see you just smile\nI want you to prosper and come proper\nEven if that means I ain t by your side\nDevil on my shoulder tellin  me I ll die soon\nI don t really want that to impact you\nBut I don t know  maybe I m just paranoid\nI just want the best for you  I just want what s left of you\nOh  you tell me that you re sad inside\nI m sad that I can t satisfy\nYeah  I pray that I get it right this time\nMaybe we ll be alright\nOh  you tell me that you re sad inside\nI m sad that I can t satisfy\nYeah  I pray that I get it right this time\nMaybe we ll be alright\nI d do anything in my power to see you just smile\nI want you to prosper and come proper\nEven if that means I ain t by your side\nI d do anything in my power to see you just smile\nI want you to prosper and come proper\nEven if that means I ain t by your side\nOh  yea

In [100]:
lyrics_eng_list = []
for i in top_five_df['url']:
    lyrics_eng_list.append(lyrics[i== lyrics['url']]['lyrics_english'].values[0])
for i in top_five_df_1['url']:
    lyrics_eng_list.append(lyrics[i== lyrics['url']]['lyrics_english'].values[0])

In [101]:
lyrics_eng_list

['I d do anything in my power to see you just smile\nI want you to prosper and come proper\nEven if that means I ain t by your side\nI d do anything in my power to see you just smile\nI want you to prosper and come proper\nEven if that means I ain t by your side\nDevil on my shoulder tellin  me I ll die soon\nI don t really want that to impact you\nBut I don t know  maybe I m just paranoid\nI just want the best for you  I just want what s left of you\nOh  you tell me that you re sad inside\nI m sad that I can t satisfy\nYeah  I pray that I get it right this time\nMaybe we ll be alright\nOh  you tell me that you re sad inside\nI m sad that I can t satisfy\nYeah  I pray that I get it right this time\nMaybe we ll be alright\nI d do anything in my power to see you just smile\nI want you to prosper and come proper\nEven if that means I ain t by your side\nI d do anything in my power to see you just smile\nI want you to prosper and come proper\nEven if that means I ain t by your side\nOh  ye